In [1]:
import os
import time
import numpy as np
import pandas as pd
import pickle as pkl
import pyomo.environ as pyomo
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

solver_kwargs={'_name':'cbc','executable':'src/cbc'}

In [7]:
deep_reload(src)

vol=100
abv=0.04

#Data on blending constituents
data={
'A':{'abv':0.045,'cost':0.32},
'B':{'abv':0.037,'cost':0.25},
'W':{'abv':0.000,'cost':0.05},
}

model=src.optimization.BlendingProblemExample(vol,abv,data,solver_kwargs)

print('Optimal Blend')
for c in data.keys():
    print('  ', c, ':', model.x[c](), 'gallons')
print()
print('Volume = ', model.volume(), 'gallons')
print('Cost = $', model.cost())

Optimal Blend
   A : 37.5 gallons
   B : 62.5 gallons
   W : 0.0 gallons

Volume =  100.0 gallons
Cost = $ 27.625


In [94]:
np.tile(inputs['f_g_min'],(m,1)).T.shape

(5, 24)

In [129]:
deep_reload(src)
'''
reservoirs will be said to have around 10 hydro units of capacity
reservoir lower bounds will be around 5 hydro units
reservoir upper bounds will be around 9 hydro units
generator and pump operating ranges will be in the range of 1 to 2 hydro unit per hour
generator cost will be around -1 power units per hydro unit per hour
generator cost will be around 1.2 power units per hydro unit per hour
transmission cost will be 1 for now
rainfall will be around .1 hydro units per hour
power demand will be around 1 power units per hour
'''


n=5 #reservoirs
m=24 #hours

inputs={
    'N':n,
    'M':m,
    'c_g':-(np.ones((n,m))*1+np.random.rand(n,m)*0),
    'c_p':np.ones((n,m))*1.2+np.random.rand(n,m)*0,
    'c_tp':np.ones((m,))*1+np.random.rand(m)*0,
    'c_ts':-(np.ones((m,))*.8+np.random.rand(m)*0),
    'y_e':np.ones((m,))*1+np.random.rand(m)*0,
    'y_f':np.ones((n,m))*.5+np.random.rand(n,m)*0,
    'lb':np.ones((n,))*5+np.random.rand(n)*0,
    'ub':np.ones((n,))*9+np.random.rand(n)*0,
    'fs':np.ones((n,))*7+np.random.rand(n)*0,
    'is':np.ones((n,))*7+np.random.rand(n)*0,
    'f_g_min':np.ones((n,))*1+np.random.rand(n)*0,
    'f_g_max':np.ones((n,))*2+np.random.rand(n)*0,
    'f_p_min':np.ones((n,))*1+np.random.rand(n)*0,
    'f_p_max':np.ones((n,))*2+np.random.rand(n)*0,
    'tp_max':10,
    'ts_max':10,
}

t0=time.time()
model=src.optimization.CHRP(inputs)

t1=time.time()
solver=pyomo.SolverFactory(**solver_kwargs)
solver.solve(model)

t2=time.time()

print(model.objective(),t1-t0,t2-t1)

24.0 0.01973128318786621 0.04637336730957031


In [115]:
model.solutions.__dict__

{'_instance': <weakref at 0x7fd0947015d0; to 'ConcreteModel' at 0x7fd0947c27b0>,
 'symbol_map': {},
 'solutions': [<pyomo.core.base.PyomoModel.ModelSolution at 0x7fd09058ce10>],
 'index': 0}

In [128]:
model.u_tp.pprint()

u_tp : Size=24, Index=T
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :   0.0 :    10 : False : False :  Reals
      1 :     0 :   0.0 :    10 : False : False :  Reals
      2 :     0 :  10.0 :    10 : False : False :  Reals
      3 :     0 :  10.0 :    10 : False : False :  Reals
      4 :     0 :   0.0 :    10 : False : False :  Reals
      5 :     0 :   0.0 :    10 : False : False :  Reals
      6 :     0 :   0.0 :    10 : False : False :  Reals
      7 :     0 :   0.0 :    10 : False : False :  Reals
      8 :     0 :   0.0 :    10 : False : False :  Reals
      9 :     0 :   0.0 :    10 : False : False :  Reals
     10 :     0 :   0.0 :    10 : False : False :  Reals
     11 :     0 :   0.0 :    10 : False : False :  Reals
     12 :     0 :   0.0 :    10 : False : False :  Reals
     13 :     0 :   0.0 :    10 : False : False :  Reals
     14 :     0 :   0.0 :    10 : False : False :  Reals
     15 :     0 :   0.0 :    10 : False : False :  Reals
     16

In [58]:
a=[1]
b=a
a[0]+=1
print(b)

[2]


In [59]:
a=np.array([1])
b=a
a[0]+=1
print(b)

[2]


In [60]:
a=np.ones((24,34))
len(a)

24

In [68]:
from dis import dis

def fun():
    return list(1)
dis(fun)

  3           0 RESUME                   0

  4           2 LOAD_GLOBAL              1 (NULL + list)
             14 LOAD_CONST               1 (1)
             16 PRECALL                  1
             20 CALL                     1
             30 RETURN_VALUE


In [69]:
from dis import dis

def fun():
    return [1]
dis(fun)

  3           0 RESUME                   0

  4           2 LOAD_CONST               1 (1)
              4 BUILD_LIST               1
              6 RETURN_VALUE
